In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [5]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [9]:

from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

170499072it [00:32, 5291536.04it/s]                               
Extracting ../data-unversioned/p1ch6/cifar-10-python.tar.gz to ../data-unversioned/p1ch6/
Files already downloaded and verified


In [10]:
label_map = {6: 0, 7: 1, 8: 2, 9: 3}
class_names = ['frog', 'horse', 'ship', 'truck']
cifar4 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [6, 7, 8, 9]]
cifar4_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [6, 7, 8, 9]]

In [12]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 4)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8) # <1>
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [17]:

import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [22]:
import time
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

train_loader = torch.utils.data.DataLoader(cifar4, batch_size=64,
                                           shuffle=True)

model = Net().to(device=device)  # <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

time1 = time.time()
training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
time2 = time.time()
time3 = time2-time1
print(time3)

Training on device cuda.
2021-04-19 18:18:20.296050 Epoch 1, Training loss 1.1301657207095965
2021-04-19 18:18:27.401526 Epoch 10, Training loss 0.5204326878911771
2021-04-19 18:18:34.936812 Epoch 20, Training loss 0.35997181154858954
2021-04-19 18:18:42.466178 Epoch 30, Training loss 0.29322803825044785
2021-04-19 18:18:49.931627 Epoch 40, Training loss 0.24757534660660802
2021-04-19 18:18:57.363323 Epoch 50, Training loss 0.21553016941958722
2021-04-19 18:19:04.804121 Epoch 60, Training loss 0.18973186076544343
2021-04-19 18:19:12.334651 Epoch 70, Training loss 0.16738538921307833
2021-04-19 18:19:19.691568 Epoch 80, Training loss 0.1494107089770107
2021-04-19 18:19:27.102634 Epoch 90, Training loss 0.13331729198654238
2021-04-19 18:19:34.448802 Epoch 100, Training loss 0.1193500435449921
2021-04-19 18:19:41.860092 Epoch 110, Training loss 0.10669211797511426
2021-04-19 18:19:49.228394 Epoch 120, Training loss 0.09267599922351943
2021-04-19 18:19:56.686352 Epoch 130, Training loss 0.

In [23]:
train_loader = torch.utils.data.DataLoader(cifar4, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar4_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

all_acc_dict["baseline"] = validate(model, train_loader, val_loader)

Accuracy train: 0.99
Accuracy val: 0.90
